In [12]:
import pandas as pd
import sqlite3
import psycopg2
from dotenv import load_dotenv, find_dotenv
import os
import sqlite3

In [5]:
load_dotenv(find_dotenv())

True

In [8]:
dbname = os.environ.get('dbanme')
user = os.environ.get('user')
password = os.environ.get('password')
host = os.environ.get('host')

In [154]:
pg_conn = psycopg2.connect(dbname = dbname, user = user, password = password, host = host)
pg_curs = pg_conn.cursor()

In [13]:
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public',
  'charactercreator_character',
  'hyedsfxe',
  None,
  True,
  False,
  False,
  False),
 ('public', 'titanic_data', 'hyedsfxe', None, True, False, False, False)]

## Titanic Dataset

### How many passengers survived, how many died

In [37]:
def get_query(query, info_str, curs):
    curs.execute(query)
    print(info_str, curs.fetchall())

In [41]:
get_query('SELECT COUNT(*) FROM titanic_data', 'All', pg_curs)

All [(887,)]


In [39]:
query = 'SELECT COUNT(Survived) FROM titanic_data WHERE Survived = 1 '
get_query(query, 'Survived', pg_curs)
query = 'SELECT COUNT(Survived) FROM titanic_data WHERE Survived = 0 '
get_query(query, 'Died', pg_curs)

Survived [(342,)]
Died [(545,)]


### How many passengers were in each class?

In [50]:
# Count of the class and group by the class

query = 'SELECT COUNT(Pclass), Pclass FROM titanic_data GROUP BY Pclass'
get_query(query, 'Count of passengers in each class', pg_curs)

Count of passengers in each class [(216, 1), (487, 3), (184, 2)]


### How many passengers survived/died in each class?

In [66]:
# Count of survivors where survived = 0/1 and group by pclass

query = 'SELECT COUNT(Survived), Pclass FROM titanic_data GROUP BY Pclass, Survived ORDER BY Pclass, Survived DESC'
get_query(query, 'Count of those who survived in each class vs those who died in each class', pg_curs)

Count of those who survived in each class vs those who died in each class [(136, 1), (80, 1), (87, 2), (97, 2), (119, 3), (368, 3)]


### What was the average age of survivors vs nonsurvivors

In [93]:
# Average age grouped by Survived

query = 'SELECT Survived, AVG(Age) FROM titanic_data GROUP BY Survived'
get_query(query, 'Average age of survivors vs non survivors', pg_curs)

Average age of survivors vs non survivors [(0, 30.1385321100917), (1, 28.4083918128272)]


### What was the average age of each passenger class?

In [109]:
# Average age grouped by class

query = 'SELECT Pclass, AVG(Age) FROM titanic_data GROUP BY Pclass ORDER BY Pclass ASC'
get_query(query, 'Average age of each class', pg_curs )

Average age of each class [(1, 38.7889814815587), (2, 29.8686413042571), (3, 25.188747433238)]


### What was the average fare of each passenger by class and survival?

In [116]:
# Average fare grouped by class and survived

query = 'SELECT Pclass, Survived, AVG(Fare) FROM titanic_data GROUP BY Pclass, Survived ORDER BY Pclass DESC, Survived DESC'
get_query(query, 'Average fare of each passenger by class and survival', pg_curs)

Average fare of each passenger by class and survival [(3, 1, 13.694887394958), (3, 0, 13.7118529891304), (2, 1, 22.0557), (2, 0, 19.4123278350515), (1, 1, 95.6080286764706), (1, 0, 64.6840075)]


### How many siblings/spouses are aboard on average by passenger class/survival?

In [128]:
# Average Sib/Spouses Group by passenger class/survival

query = "SELECT Pclass, Survived, AVG(Siblings_Spouses_Aboard) FROM titanic_data GROUP BY Pclass, Survived ORDER BY Survived DESC, Pclass DESC"
get_query(query, 'Siblings/spouses on average per class and survived', pg_curs)

Siblings/spouses on average per class and survived [(3, 1, Decimal('0.43697478991596638655')), (2, 1, Decimal('0.49425287356321839080')), (1, 1, Decimal('0.49264705882352941176')), (3, 0, Decimal('0.67934782608695652174')), (2, 0, Decimal('0.31958762886597938144')), (1, 0, Decimal('0.28750000000000000000'))]


In [129]:
# Average Parents/Children Group by passenger class/survival

query = 'SELECT Pclass, Survived, AVG(Parents_Children_Aboard) FROM titanic_data GROUP BY Pclass, Survived ORDER BY Survived DESC, Pclass DESC'
get_query(query, 'Parents/children on average per class and survived', pg_curs)

Parents/children on average per class and survived [(3, 1, Decimal('0.42016806722689075630')), (2, 1, Decimal('0.64367816091954022989')), (1, 1, Decimal('0.38970588235294117647')), (3, 0, Decimal('0.38858695652173913043')), (2, 0, Decimal('0.14432989690721649485')), (1, 0, Decimal('0.30000000000000000000'))]


In [155]:
# Any Duplicate Names?

query = 'SELECT Name FROM titanic_data GROUP BY Name HAVING COUNT(Name) > 1'
get_query(query, '', pg_curs)

 []
